<a href="https://colab.research.google.com/github/andrea212023/ML-Kis/blob/main/HW_2_4_Boosting_algorithms_KIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням алгоритмів бустингу: XGBoost та LightGBM, а також використаємо бібліотеку HyperOpt для оптимізації гіперпараметрів.

0. Зчитайте дані `train.csv` в змінну `raw_df` та скористайтесь наведеним кодом нижче аби розділити дані на трнувальні та валідаційні і розділити дані на ознаки з матириці Х та цільову змінну. Назви змінних `train_inputs, train_targets, train_inputs, train_targets` можна змінити на ті, які Вам зручно.

  Наведений скрипт - частина отриманого мною скрипта для обробки даних. Ми тут не викнуємо масштабування та обробку категоріальних змінних, бо хочемо це делегувати алгоритмам, які будемо використовувати. Якщо щось не розумієте в наведених скриптах, рекомендую розібратись: навичка читати код - важлива складова роботи в машинному навчанні.

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
!pip install opendatasets --upgrade --quiet
!pip install hyperopt --quiet

In [45]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [46]:
import opendatasets as od
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, classification_report
from typing import Tuple, Dict, Any

In [47]:
dataset_url = 'https://www.kaggle.com/competitions/bank-customer-churn-prediction-dlu/'
od.download(dataset_url)
data_dir = './bank-customer-churn-prediction-dlu'
raw_df = pd.read_csv('./bank-customer-churn-prediction-dlu/train.csv')

Skipping, found downloaded files in "./bank-customer-churn-prediction-dlu" (use force=True to force download)


In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from typing import Tuple, Dict, Any


def split_train_val(df: pd.DataFrame, target_col: str, test_size: float = 0.2, random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split the dataframe into training and validation sets.

    Args:
        df (pd.DataFrame): The raw dataframe.
        target_col (str): The target column for stratification.
        test_size (float): The proportion of the dataset to include in the validation split.
        random_state (int): Random state for reproducibility.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: Training and validation dataframes.
    """
    train_df, val_df = train_test_split(df, test_size=test_size, random_state=random_state, stratify=df[target_col])
    return train_df, val_df


def separate_inputs_targets(df: pd.DataFrame, input_cols: list, target_col: str) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Separate inputs and targets from the dataframe.

    Args:
        df (pd.DataFrame): The dataframe.
        input_cols (list): List of input columns.
        target_col (str): Target column.

    Returns:
        Tuple[pd.DataFrame, pd.Series]: DataFrame of inputs and Series of targets.
    """
    inputs = df[input_cols].copy()
    targets = df[target_col].copy()
    return inputs, targets

In [49]:
target_column = 'Exited'
input_columns = raw_df.columns.difference([target_column]).tolist()

train_df, val_df = split_train_val(raw_df, target_col=target_column)
train_inputs, train_targets = separate_inputs_targets(train_df, input_cols=input_columns, target_col=target_column)
val_inputs, val_targets = separate_inputs_targets(val_df, input_cols=input_columns, target_col=target_column)

# Print results
print("Train inputs:")
print(train_inputs.head())
print("Train targets:")
print(train_targets.head())
print("Validation inputs:")
print(val_inputs.head())
print("Validation targets:")
print(val_targets.head())

Train inputs:
        Age    Balance  CreditScore  CustomerId  EstimatedSalary  Gender  \
7180   30.0  131394.56        682.0  15652218.0        143952.24    Male   
10393  39.0  178058.06        684.0  15592937.0        145518.31  Female   
80     35.0  116320.68        705.0  15774586.0        174431.01    Male   
3365   58.0       0.00        669.0  15780572.0         51565.98    Male   
12236  21.0       0.00        707.0  15642099.0        148564.76    Male   

      Geography  HasCrCard  IsActiveMember  NumOfProducts    Surname  Tenure  \
7180     France        1.0             1.0            1.0       Mays     1.0   
10393    France        1.0             0.0            1.0     Ch'eng     2.0   
80      Germany        1.0             0.0            2.0      Ch'in     6.0   
3365      Spain        0.0             1.0            2.0         K?     0.0   
12236    France        1.0             1.0            2.0  Trevisani     3.0   

          id  
7180    7180  
10393  10393  
80 

1. В тренувальному та валідаційному наборі перетворіть категоріальні ознаки на тип `category`. Можна це зробити двома способами:
 1. `df[col_name].astype('category')`, як було продемонстровано в лекції
 2. використовуючи метод `pd.Categorical(df[col_name])`

In [50]:
categorical_columns = raw_df.select_dtypes(include=['object']).columns.tolist()
categorical_columns

['Surname', 'Geography', 'Gender']

In [51]:
for col in categorical_columns:
    train_inputs[col] = pd.Categorical(train_inputs[col])
    val_inputs[col] = pd.Categorical(val_inputs[col])

# Print resulrs
print("Train inputs:")
print(train_inputs.dtypes)
print("Validation inputs:")
print(val_inputs.dtypes)

Train inputs:
Age                 float64
Balance             float64
CreditScore         float64
CustomerId          float64
EstimatedSalary     float64
Gender             category
Geography          category
HasCrCard           float64
IsActiveMember      float64
NumOfProducts       float64
Surname            category
Tenure              float64
id                    int64
dtype: object
Validation inputs:
Age                 float64
Balance             float64
CreditScore         float64
CustomerId          float64
EstimatedSalary     float64
Gender             category
Geography          category
HasCrCard           float64
IsActiveMember      float64
NumOfProducts       float64
Surname            category
Tenure              float64
id                    int64
dtype: object


2. Навчіть на отриманих даних модель `XGBoostClassifier`. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів XGBoostClassifier - тут https://xgboost.readthedocs.io/en/stable/parameter.html#global-config

  **Важливо:** зробіть такі налаштування `XGBoostClassifier` аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Можна також, якщо працюєте в Google Colab, увімкнути можливість використання GPU (`Runtime -> Change runtime type -> T4 GPU`) і встановити параметр `device='cuda'` в `XGBoostClassifier` для пришвидшення тренування бустинг моделі.
  
  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням DecisionTrees раніше. Чи вийшло покращити якість?

In [52]:
# Implement XGBClassifier algorithm
xgb_clf = XGBClassifier(
    use_label_encoder=False,
    missing=np.nan,
    enable_categorical=True,
    random_state=42,
    device='cuda',
    max_depth=3,
    n_estimators=10
)

xgb_clf.fit(train_inputs, train_targets)

# Predict on train and validation datasets
train_preds = xgb_clf.predict_proba(train_inputs)[:, 1]
val_preds = xgb_clf.predict_proba(val_inputs)[:, 1]

# Compute AUROC for train and validation datasets
train_auc = roc_auc_score(train_targets, train_preds)
val_auc = roc_auc_score(val_targets, val_preds)

# Print AUROC for train and validation datasets
print(f"AUROC for train dataset: {train_auc}")
print(f"AUROC for validation dataset: {val_auc}")

AUROC for train dataset: 0.9421334105891545
AUROC for validation dataset: 0.9251563893271144


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:38:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [53]:
# Define train and validation inputs
train_pred = xgb_clf.predict(train_inputs)
val_pred = xgb_clf.predict(val_inputs)

# Print classification reports
print(classification_report(train_targets, train_pred, digits=4))
print(classification_report(val_targets, val_pred, digits=4))

              precision    recall  f1-score   support

         0.0     0.9162    0.9684    0.9416      9558
         1.0     0.8408    0.6532    0.7352      2442

    accuracy                         0.9042     12000
   macro avg     0.8785    0.8108    0.8384     12000
weighted avg     0.9008    0.9042    0.8996     12000

              precision    recall  f1-score   support

         0.0     0.9047    0.9653    0.9340      2390
         1.0     0.8156    0.6016    0.6925       610

    accuracy                         0.8913      3000
   macro avg     0.8601    0.7835    0.8132      3000
weighted avg     0.8866    0.8913    0.8849      3000



3. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `XGBoostClassifier` з лекції знайдіть оптимальні значення гіперпараметрів `XGBoostClassifier` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **20**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. В ній ми маємо задати loss - це може будь-яка метрика, але бажано використовувтаи ту, яка цільова в вашій задачі. Чим менший лосс - тим ліпша модель на думку hyperopt. Тож, тут нам треба задати loss - негативне значення AUROC. В лекції ми натомість використовували Accuracy.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_clf` модель `XGBoostClassifier` з найкращими гіперпараметрами
    - навчіть модель `final_clf`
    - оцініть якість моделі `final_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (2) цього завдання?

In [60]:
# Objective function for hyperparameter tuning
def objective(params):
    clf = xgb.XGBClassifier(
        n_estimators=int(params['n_estimators']),
        learning_rate=params['learning_rate'],
        max_depth=int(params['max_depth']),
        min_child_weight=params['min_child_weight'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        enable_categorical=True,  # Enable categorical handling
        use_label_encoder=False,
        tree_method='gpu_hist' if params.get('device') == 'cuda' else 'hist',  # Use GPU if available
        eval_metric='auc'  # Evaluation metric is now specified here
    )

    # Fit the model
    clf.fit(
        train_inputs,
        train_targets,
        eval_set=[(val_inputs, val_targets)],  # Validation set for evaluation
        verbose=False
    )

    # Predict probabilities
    pred = clf.predict_proba(val_inputs)[:, 1]

    # Calculate validation AUC
    auc = roc_auc_score(val_targets, pred)

    # Return the negative AUC as Hyperopt minimizes the objective
    return {'loss': -auc, 'status': STATUS_OK}

# Define the parameter search space
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 25),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'max_depth': hp.quniform('max_depth', 3, 15, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'gamma': hp.uniform('gamma', 0, 0.5),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),
    'device': hp.choice('device', ['cuda', 'cpu'])  # Use GPU or CPU
}

# Perform optimization
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=20, trials=trials)

# Convert integer parameters back to int
best['n_estimators'] = int(best['n_estimators'])
best['max_depth'] = int(best['max_depth'])
best['min_child_weight'] = int(best['min_child_weight'])

# Print best parameters
print("Best hyperparameters: ", best)


  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:42:56] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:42:56] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



  5%|▌         | 1/20 [00:04<01:17,  4.07s/trial, best loss: -0.9248391522052267]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:43:00] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:43:00] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:43:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



 10%|█         | 2/20 [00:06<00:54,  3.03s/trial, best loss: -0.9248391522052267]

...

 90%|█████████ | 18/20 [00:46<00:02,  1.41s/trial, best loss: -0.9349677618492351]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:43:42] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:43:42] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:43:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



 95%|█████████▌| 19/20 [00:47<00:01,  1.42s/trial, best loss: -0.9349677618492351]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:43:43] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:43:43] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



100%|██████████| 20/20 [00:48<00:00,  2.44s/trial, best loss: -0.9349677618492351]
Best hyperparameters:  {'colsample_bytree': 0.596204237050998, 'device': 1, 'gamma': 0.2831238052385673, 'learning_rate': 0.06803284752535987, 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 100, 'reg_alpha': 0.17656031881168488, 'reg_lambda': 0.18740985864864013, 'subsample': 0.8900034320657852}


In [61]:
best

{'colsample_bytree': 0.596204237050998,
 'device': 1,
 'gamma': 0.2831238052385673,
 'learning_rate': 0.06803284752535987,
 'max_depth': 3,
 'min_child_weight': 2,
 'n_estimators': 100,
 'reg_alpha': 0.17656031881168488,
 'reg_lambda': 0.18740985864864013,
 'subsample': 0.8900034320657852}

In [63]:
# Train the final model with the best hyperparameters
final_clf = xgb.XGBClassifier(
    n_estimators=best['n_estimators'],
    learning_rate=best['learning_rate'],
    max_depth=best['max_depth'],
    min_child_weight=best['min_child_weight'],
    subsample=best['subsample'],
    colsample_bytree=best['colsample_bytree'],
    gamma=best['gamma'],
    reg_alpha=best['reg_alpha'],
    reg_lambda=best['reg_lambda'],
    enable_categorical=True,
    use_label_encoder=False,
    missing=np.nan,
    tree_method='hist',  # Use CPU (or 'gpu_hist' for GPU if applicable)
    eval_metric="auc"  # Built-in evaluation metric
)

# Train the model
final_clf.fit(
    train_inputs,
    train_targets,
    eval_set=[(val_inputs, val_targets)],
    verbose=False
)

# Predict probabilities for train and validation datasets
train_preds_proba = final_clf.predict_proba(train_inputs)[:, 1]
val_preds_proba = final_clf.predict_proba(val_inputs)[:, 1]

# Calculate AUROC for train and validation datasets
train_auc = roc_auc_score(train_targets, train_preds_proba)
val_auc = roc_auc_score(val_targets, val_preds_proba)

print(f"Train AUROC: {train_auc:.4f}")
print(f"Validation AUROC: {val_auc:.4f}")

# Predict class labels for train and validation datasets
train_preds = final_clf.predict(train_inputs)
val_preds = final_clf.predict(val_inputs)

# Print classification reports
print("Classification Report (Train):")
print(classification_report(train_targets, train_preds, digits=4))

print("Classification Report (Validation):")
print(classification_report(val_targets, val_preds, digits=4))


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:59:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Train AUROC: 0.9472
Validation AUROC: 0.9350
Classification Report (Train):
              precision    recall  f1-score   support

         0.0     0.9226    0.9669    0.9443      9558
         1.0     0.8406    0.6826    0.7534      2442

    accuracy                         0.9091     12000
   macro avg     0.8816    0.8248    0.8489     12000
weighted avg     0.9059    0.9091    0.9054     12000

Classification Report (Validation):
              precision    recall  f1-score   support

         0.0     0.9217    0.9502    0.9357      2390
         1.0     0.7780    0.6836    0.7277       610

    accuracy                         0.8960      3000
   macro avg     0.8498    0.8169    0.8317      3000
weighted avg     0.8925    0.8960    0.8934      3000



**Observations:**

Overall Performance:

Last model (with best hyperparameters) performs slightly better in both train and validation sets, as evidenced by its higher AUROC and weighted average F1-scores.

Class Imbalance Handling:

Last model achieves better recall and F1-scores for the minority class (Class 1) on both the training and validation sets compared to original model. This indicates that Model 1 is better at handling the minority class.

Precision for Class 0:

Both models have high precision for Class 0, but last model is marginally better.

Recall for Class 1:

Model 1 has significantly higher recall for the minority class (Train: 0.6826 vs. 0.6016; Validation: 0.6532 vs. 0.6016). This suggests last model is better at identifying positive cases.

Accuracy:

Last model achieves higher accuracy on both the training and validation sets (Train: 0.9091 vs. 0.8913; Validation: 0.9042 vs. 0.8913).

Weighted F1-Score:

Last model has a higher weighted F1-score on both train and validation sets, demonstrating superior overall performance.

**Conclusion:**

Last model (with best hyperparameters) is the better-performing model. It achieves higher AUROC, weighted F1-scores, and better recall for the minority class, making it more suitable for imbalanced datasets.
Original model, while still reasonable, falls short, particularly in handling the minority class, which is critical in imbalanced classification problems.

4. Навчіть на наших даних модель LightGBM. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів LightGBM - тут https://lightgbm.readthedocs.io/en/latest/Parameters.html

  **Важливо:** зробіть такі налаштування LightGBM аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Аби передати категоріальні колонки в LightGBM - необхідно виявити їх індекси і передати в параметрі `cat_feature=cat_feature_indexes`

  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням XGBoostClassifier раніше. Чи вийшло покращити якість?

In [64]:
import lightgbm as lgb
print(lgb.__version__)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


4.5.0


In [65]:
cat_feature_indexes = [train_inputs.columns.get_loc(col) for col in categorical_columns]

In [68]:
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from sklearn.metrics import roc_auc_score, classification_report

# Define the LightGBM Classifier with specified hyperparameters
lgb_clf = lgb.LGBMClassifier(
    max_depth=3,
    n_estimators=50,
    learning_rate=0.1,
    categorical_feature=cat_feature_indexes,  # For recognizing categorical features
    missing=np.nan,  # Explicitly specify missing values
    device_type='cpu'  # Explicitly set the device type (change to 'gpu' if GPU is available)
)

# Train the model with a validation set using callbacks
lgb_clf.fit(
    train_inputs,
    train_targets,
    eval_set=[(val_inputs, val_targets)],  # Validation set for evaluation
    eval_metric='auc',  # Use AUC for evaluation
    callbacks=[
        early_stopping(stopping_rounds=10),  # Stop if no improvement after 10 rounds
        log_evaluation(period=10)  # Log evaluation metrics every 10 rounds
    ]
)

# Predict probabilities for train and validation datasets
train_preds_proba = lgb_clf.predict_proba(train_inputs)[:, 1]
val_preds_proba = lgb_clf.predict_proba(val_inputs)[:, 1]

# Calculate AUROC for train and validation datasets
train_auc = roc_auc_score(train_targets, train_preds_proba)
val_auc = roc_auc_score(val_targets, val_preds_proba)

print(f"Train AUROC: {train_auc:.4f}")
print(f"Validation AUROC: {val_auc:.4f}")

# Predict class labels for train and validation datasets
train_preds = lgb_clf.predict(train_inputs)
val_preds = lgb_clf.predict(val_inputs)

# Print classification reports
print("Classification Report (Train):")
print(classification_report(train_targets, train_preds, digits=4))

print("Classification Report (Validation):")
print(classification_report(val_targets, val_preds, digits=4))


/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(


[LightGBM] [Warning] Unknown parameter: missing
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] Unknown parameter: missing
[LightGBM] [Warning] categorical_feature is set=10,6,5, categorical_column=5,6,10 will be ignored. Current value: categorical_feature=10,6,5
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001911 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1826
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 13
[LightGBM] [Warning] Unknown parameter: missing
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> in

**Observations:**

**Strengths of LightGBM model:**

1. High AUROC: Both Train and Validation AUROC scores are very high (0.9496 and 0.9350), showing strong discriminatory power between classes.
2. Good Performance on Majority Class:
For Class 0, precision, recall, and F1-scores are consistently high across both train and validation datasets.
3. Generalization:
The validation performance is close to the training performance, indicating that the model generalizes well without overfitting.

**Weaknesses of LightGBM model:**
1. Class Imbalance Impact:
The minority class (Class 1) has noticeably lower recall (Train: 0.6884, Validation: 0.6572), indicating that the model struggles to identify some positive cases.
This issue is common in imbalanced datasets and affects the F1-score for the minority class.

**Conclusions:**

1. The model performs exceptionally well for the majority class and achieves high overall AUROC and weighted F1-scores, making it a reliable model for general tasks.
2. However, the recall for the minority class is lower, which could be problematic if the task requires high sensitivity for Class 1.
3. To improve minority class performance, techniques like class weighting, oversampling (e.g., SMOTE), or advanced algorithms designed to handle imbalanced datasets could be considered.


**Recommendations:**
1. Fine-tune for Minority Class:
Use class_weight='balanced' or custom weights in the LightGBM classifier to penalize errors on the minority class.
2. Additional Feature Engineering:
Add or transform features to better distinguish between the two classes, potentially improving recall for the minority class.
3. Resampling Techniques:
Oversample the minority class using SMOTE or other techniques to provide the model with more balanced training data.
4. Hyperparameter Optimization:
Perform grid search or randomized search to fine-tune hyperparameters further, especially those affecting regularization (e.g., reg_alpha, reg_lambda).

5. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `LightGBM` з лекції знайдіть оптимальні значення гіперпараметрів `LightGBM` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **10**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. І тут ми також ставимо loss - негативне значення AUROC, як і при пошуці гіперпараметрів для XGBoost. До речі, можна спробувати написати код так, аби в objective передавати лише модель і не писати схожий код двічі :)

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_lgb_clf` модель `LightGBM` з найкращими гіперпараметрами
    - навчіть модель `final_lgb_clf`
    - оцініть якість моделі `final_lgb_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (4) цього завдання?

In [69]:
# Define the objective function
def objective(params, train_inputs, train_targets, val_inputs, val_targets):
    # Ensure num_leaves >= 2 ** max_depth
    params['num_leaves'] = max(int(params['num_leaves']), 2 ** int(params['max_depth']))

    clf = lgb.LGBMClassifier(
        n_estimators=int(params['n_estimators']),
        learning_rate=params['learning_rate'],
        max_depth=int(params['max_depth']),
        num_leaves=int(params['num_leaves']),
        min_child_weight=params['min_child_weight'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        max_bin=int(params['max_bin']),
        categorical_feature=params['cat_feature'],
        missing=np.nan  # Explicitly handle missing values
    )

    clf.fit(
        train_inputs,
        train_targets,
        eval_set=[(val_inputs, val_targets)],
        eval_metric='auc',
        callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=False)]
    )

    # Predict probabilities and compute validation AUC
    val_preds = clf.predict_proba(val_inputs)[:, 1]
    auc = roc_auc_score(val_targets, val_preds)

    return {'loss': -auc, 'status': STATUS_OK}

# Define the hyperparameter search space
param_space = {
    'num_leaves': hp.choice('num_leaves', range(31, 151, 10)),
    'max_depth': hp.choice('max_depth', range(0, 15, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),  # log-uniform for smaller values
    'n_estimators': hp.choice('n_estimators', range(50, 501, 50)),
    'min_child_weight': hp.uniform('min_child_weight', 1, 10),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'max_bin': hp.choice('max_bin', range(100, 256, 10)),
    'cat_feature': hp.choice('cat_feature', [cat_feature_indexes])  # Pass categorical feature indexes
}

# Run optimization
trials = Trials()
best_params = fmin(
    fn=lambda params: objective(params, train_inputs, train_targets, val_inputs, val_targets),
    space=param_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=trials
)

# Convert hyperparameters to correct types
best_params['num_leaves'] = int(best_params['num_leaves'])
best_params['max_depth'] = int(best_params['max_depth'])
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['max_bin'] = int(best_params['max_bin'])

# Ensure num_leaves >= 2 ** max_depth
best_params['num_leaves'] = max(best_params['num_leaves'], 2 ** best_params['max_depth'])

print("Best Hyperparameters: ", best_params)


[LightGBM] [Warning] Unknown parameter: missing
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] Unknown parameter: missing
[LightGBM] [Warning] categorical_feature is set=10,6,5, categorical_column=5,6,10 will be ignored. Current value: categorical_feature=10,6,5
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002443 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1706
  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(

/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")



[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 13
[LightGBM] [Warning] Unknown parameter: missing
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(

/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(

/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")



[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 13
[LightGBM] [Warning] Unknown parameter: missing
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
 90%|█████████ | 9/10 [00:10<00:01,  1.15s/trial, best loss: -0.9377824267782428]

/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(

/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [70]:
# Ensure hyperparameters are logical
if best_params['n_estimators'] < 10:
    best_params['n_estimators'] = 50
if best_params['max_bin'] <= 0:
    best_params['max_bin'] = 255

# Create the final LightGBM model with the best hyperparameters
final_lgb_clf = lgb.LGBMClassifier(
    num_leaves=best_params['num_leaves'],
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    n_estimators=best_params['n_estimators'],
    min_child_weight=best_params['min_child_weight'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    reg_alpha=best_params['reg_alpha'],
    reg_lambda=best_params['reg_lambda'],
    max_bin=best_params['max_bin']
)

# Train the model
final_lgb_clf.fit(train_inputs, train_targets)

# Evaluate on the training set
train_pred_proba = final_lgb_clf.predict_proba(train_inputs)[:, 1]
train_pred = (train_pred_proba > 0.5).astype(int)
train_roc_auc = roc_auc_score(train_targets, train_pred_proba)

# Evaluate on the validation set
val_pred_proba = final_lgb_clf.predict_proba(val_inputs)[:, 1]
val_pred = (val_pred_proba > 0.5).astype(int)
val_roc_auc = roc_auc_score(val_targets, val_pred_proba)

# Print AUROC scores
print(f"Train AUROC: {train_roc_auc:.4f}")
print(f"Validation AUROC: {val_roc_auc:.4f}")

# Classification report for the training set
print("\nClassification Report (Train):")
print(classification_report(train_targets, train_pred, digits=4))

# Classification report for the validation set
print("\nClassification Report (Validation):")
print(classification_report(val_targets, val_pred, digits=4))


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001489 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 572
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

**Observations:**

**Train dataset:**
1. The model achieved an AUROC of 0.9657 on the training dataset, indicating excellent discriminatory power.
2. The model performs better on Class 0 compared to Class 1, likely due to class imbalance.
3. Overall training accuracy is 92.43%, which is high and shows that the model fits the training data well.
4. The weighted average F1-Score is 0.9221, reflecting good performance while accounting for class imbalance.
5. The macro average F1-Score is slightly lower at 0.8765, showing reduced performance on the minority class (Class 1).

**Validation dataset:**
1. The model achieved an AUROC of 0.9367 on the validation dataset, showing that it generalizes well to unseen data.
2. Similar to the training dataset, the model struggles to classify the minority class (Class 1), with a noticeable drop in recall (67.21%) and F1-Score (72.44%).
3. The overall validation accuracy is 89.60%, slightly lower than the training accuracy, which indicates good generalization with limited overfitting.
4. The weighted average F1-Score is 0.8929, which is consistent with the high AUROC.
5. The macro average F1-Score is 0.8301, showing lower performance on Class 1.

**Conclusions:**

1. Overall performance: The LightGBM classifier demonstrates strong performance on both the training and validation datasets, achieving high AUROC values (0.9657 on training and 0.9367 on validation).
The slight difference between training and validation performance suggests the model generalizes well without significant overfitting.
2. Class imbalance impact: The model performs better on the majority class (Class 0) compared to the minority class (Class 1). This is evident in the lower recall and F1-Score for Class 1.
The imbalance affects the overall macro average F1-Score, particularly in the validation dataset.

**Recommendations:**

To improve performance on the minority class (Class 1), we can consider using techniques like:

1. Adjusting the decision threshold for class probabilities.
2. Trying cost-sensitive learning (e.g., adding class weights to penalize misclassifications of the minority class).
3. Experimenting with oversampling (e.g., SMOTE) or undersampling to balance the dataset.

**Final assessment:**

The model is well-suited for this dataset, but improvements in recall and F1-Score for Class 1 should be prioritized to enhance its overall effectiveness in real-world applications.

6. Оберіть модель з експериментів в цьому ДЗ і зробіть новий `submission` на Kaggle та додайте код для цього і скріншот скора на публічному лідерборді.
  
  **Напишіть коментар, чому ви обрали саме цю модель?**

  І я вас вітаю - це останнє завдання з цим набором даних 💪 На цьому етапі корисно проаналізувати, які моделі показали себе найкраще і подумати, чому.

**Conclusion:**

The LightGBM model after hyperparameter optimization (Task 5) is well-suited for this dataset because it shows the best results on the validation datasets with the highest AUROC value and well-balanced precision and recall values for both classes. This model has the best overall performance and demonstrates that hyperparameter optimization significantly improved its ability to distinguish between classes.

In [72]:
# Load the test data
test_df = pd.read_csv('./bank-customer-churn-prediction-dlu/test.csv')

# Ensure test data has the same columns as train data
train_columns = train_inputs.columns.tolist()

for col in train_columns:
    if col not in test_df.columns:
        # Fill missing columns with a reasonable default
        test_df[col] = 0  # or other default based on context

# Ensure the column order matches
test_inputs = test_df[train_columns]

# Handle categorical columns to align with train data
for col in categorical_columns:
    if col in test_inputs.columns:  # Check if the column exists in the test data
        test_inputs[col] = pd.Categorical(
            test_inputs[col],
            categories=train_inputs[col].cat.categories  # Use train data categories
        )

# Predictions using the final LightGBM model
test_preds = final_lgb_clf.predict_proba(test_inputs)[:, 1]

# Prepare submission file
if 'id' not in test_df.columns:
    test_df['id'] = range(len(test_df))  # Add default IDs if missing

submission_df = pd.DataFrame({
    'id': test_df['id'],       # Ensure ID column exists
    'Exited': test_preds       # Use model predictions
})

# Save the submission file
submission_path = 'drive/MyDrive/Colab Notebooks/data/submission_light_opt.csv'
submission_df.to_csv(submission_path, index=False)

print(f"Submission file saved to {submission_path}")


Submission file saved to drive/MyDrive/Colab Notebooks/data/submission_light_opt.csv


In [73]:
submission_df.head()

,id,Exited
0,15000,0.053399
1,15001,0.021365
2,15002,0.044127
3,15003,0.523982
4,15004,0.023533
